In [3]:
import os
os.getcwd()
os.chdir(r'/home/featurize/data/Bert/')

In [4]:
#data_collator是一个函数，负责获取样本并将它们批处理成张量
#在data_collator中可以确保每次以新的方式完成随机掩蔽。
from transformers import DataCollatorForLanguageModeling
from transformers import BertTokenizer

# 定义 vocab.txt 文件的路径
vocab_file_path = 'vocab.txt'

# 以只读模式打开文件
with open(vocab_file_path, 'r', encoding='utf-8') as file:
    # 读取文件的所有行
    lines = file.readlines()
    # 计算行数
    vocab_size = len(lines)

# 打印 vocab_size 的值
print(f"vocab_size: {vocab_size}")

tokenizer = BertTokenizer(vocab_file=vocab_file_path)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

/environment/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-31 02:41:29.015773: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 02:41:29.046051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-31 02:41:29.046077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-31 02:41:29.046982:

vocab_size: 6982


In [5]:
#初始化bert模型，参数参考讨论区代码
from transformers import BertConfig

config = BertConfig(
    vocab_size=vocab_size,
    hidden_size=512,
    intermediate_size=4 * 512,
    max_position_embeddings=512,
    num_hidden_layers=4,
    num_attention_heads=4,
    type_vocab_size=2
)
from transformers import BertForMaskedLM

model = BertForMaskedLM(config=config)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [6]:
model_size = sum(t.numel() for t in model.parameters())
print(f"Bert size: {model_size / 1000 ** 2:.1f}M parameters")

Bert size: 16.7M parameters


In [11]:
#加载和保存拼接后的文本，掉线的时候这么做
# lm_datasets.save_to_disk('./preTrain_data')
# import pandas as pd
from datasets import load_from_disk
lm_datasets=load_from_disk('./preTrain_data')


In [12]:
lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 888350
})

In [7]:
# num_samples = len(lm_datasets)
# print(f"数据集的样本数为: {num_samples}")

In [13]:
#使用GPU训练，运行这段代码
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [14]:
from transformers import Trainer, TrainingArguments



training_args = TrainingArguments(
    output_dir="preTrain_log",
    logging_strategy="steps",
    logging_steps=1000,
    save_strategy="steps",
    save_steps=6000,
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=256,
    weight_decay=0.01)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets,
    data_collator=data_collator)

# #训练并保存模型
# trainer.train()

# trainer.save_model("./model")

In [23]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval_us

In [24]:
# trainer.args.learning_rate=1e-7
trainer.args.num_train_epochs=30
#训练并保存模型
trainer.train(resume_from_checkpoint='./preTrain_log/checkpoint-66000')

There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Step,Training Loss
67000,8.980500


KeyboardInterrupt: 